Существует сервис Chatbot Arena, где разные чат-боты работающие на основе больших языковых моделей (LLM), генерируют ответы на запросы пользователей. Пользователь печатает запрос, два чат-бота предлагают свои ответы, пользователю необходимо выбрать, какой из ответов наиболее подходящий - у первого бота, второго, либо получается ничья - оба бота одинаково справились с ответами/ни один из ботов не дал подходящего ответа.

Задача: необходимо разработать модель, которая будет предсказывать, какой чат-бот предпочтут пользователи. Это поможет улучшить взаимодействие чат-ботов с людьми,сделать  их более соответствующими человеческим предпочтениям.

Начнем с загрузки необходимых библиотек и открытия тренировочного набора данных.

In [ ]:
import numpy as np
import pandas as pd

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import coo_matrix, hstack
from scipy import sparse
import xgboost as xgb

from sklearn.metrics import log_loss

In [ ]:
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV
)

RANDOM_STATE = 20824

In [ ]:
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
train_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'
test_path = '/kaggle/input/lmsys-chatbot-arena/test.csv'

df = pd.read_csv(train_path, sep=',')

В тренировочной базе несколько переменных.

**model_a, model_b** - модель бота, отвечающего на запрос. Можно выделить 16 наиболее встречающихся модели.

**prompt** - запрос, который пишет пользователь.

**response_a, response_b** - ответы ботов.

**winner_model_a, winner_model_b, winner_tie** - победа бота a, b или ничья.

Всего 57477 наблюдения.

In [ ]:
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


Добавим несколько расчетных переменных - длину запроса и ответов ботов, разницу между длиной ответов ботов, отношение длины ответов ботов к длине запрса.

In [ ]:
df['len_prompt'] = df['prompt'].apply(lambda x: len(x))
df['len_a'] = df['response_a'].apply(lambda x: len(x))
df['len_b'] = df['response_b'].apply(lambda x: len(x))

df['diff_a_b'] = (df['len_a'] - df['len_b']) / (df['len_a'])
df['len_a_prompt'] = df['len_a'] / df['len_prompt']
df['len_b_prompt'] = df['len_b'] / df['len_prompt']

Загрузим библиотеку spaCy для анализа естественного языка.

In [ ]:
# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en_core_web_sm
import spacy

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/opt/conda/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 790, in urlopen
    response = self._make_request(
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", line 491, in _make_request
    raise new_e
  File "/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py", li

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

Выделим данные с запросом пользователей (prompt) в отдельную переменную. Для каждого запроса проведем лемматизацию, удаление стоп-слов и удаление символов.  

In [ ]:
corpus_prompt = df['prompt'].values
corpus_prompt.shape

(57477,)

In [ ]:
%%time
for i in range(len(corpus_prompt)):
    corpus_prompt[i] = nlp(corpus_prompt[i])
    corpus_prompt[i] = [token for token in corpus_prompt[i] if not token.is_stop]
    corpus_prompt[i] = " ".join([token.lemma_ for token in corpus_prompt[i]])
    corpus_prompt[i] = re.sub(r"(?!'``)[\W\d]+|(?![\w])'[^a-zA-Z]", ' ', corpus_prompt[i]).lower()

CPU times: user 7min 24s, sys: 1.65 s, total: 7min 25s
Wall time: 7min 26s


То же самое сделаем для переменной с ответами бота a.

In [ ]:
corpus_a = df['response_a'].values

corpus_a.shape

(57477,)

In [ ]:
%%time
for i in range(len(corpus_a)):
    corpus_a[i] = nlp(corpus_a[i])
    corpus_a[i] = [token for token in corpus_a[i] if not token.is_stop]
    corpus_a[i] = " ".join([token.lemma_ for token in corpus_a[i]])
    corpus_a[i] = re.sub(r"(?!'``)[\W\d]+|(?![\w])'[^a-zA-Z]", ' ', corpus_a[i]).lower()

CPU times: user 18min 31s, sys: 842 ms, total: 18min 32s
Wall time: 18min 34s


То же самое сделаем для переменной с ответами бота b.

In [ ]:
corpus_b = df['response_b'].values

corpus_b.shape

(57477,)

In [ ]:
%%time
for i in range(len(corpus_b)):
    corpus_b[i] = nlp(corpus_b[i])
    corpus_b[i] = [token for token in corpus_b[i] if not token.is_stop]
    corpus_b[i] = " ".join([token.lemma_ for token in corpus_b[i]])
    corpus_b[i] = re.sub(r"(?!'``)[\W\d]+|(?![\w])'[^a-zA-Z]", ' ', corpus_b[i]).lower()

CPU times: user 18min 19s, sys: 709 ms, total: 18min 20s
Wall time: 18min 21s


Соединим обработанные запросы и ответы в один датафрейм. Объединим запрос и два ответа ботов в одну текстовую строку.

In [ ]:
corpus_prompt = pd.DataFrame(corpus_prompt)

In [ ]:
corpus_a = pd.DataFrame(corpus_a)
corpus_b = pd.DataFrame(corpus_b)

In [ ]:
corpus = pd.merge(corpus_prompt, corpus_a, left_index=True, right_index=True)

In [ ]:
corpus = pd.merge(corpus, corpus_b, left_index=True, right_index=True)

In [ ]:
corpus.columns = ['prompt', 'a', 'b']

In [ ]:
corpus['all'] = corpus['prompt'] + corpus['a'] + corpus['b']

In [ ]:
def target_column(row):
    if row['winner_model_a'] == 1:
        return 0
    if row['winner_model_b'] == 1:
        return 1
    if row['winner_tie'] == 1:
        return 2

In [ ]:
corpus['target'] = df.apply(target_column, axis=1)

In [ ]:
corpus['len_prompt'] = df['len_prompt']
corpus['len_a'] = df['len_a']
corpus['len_b'] = df['len_b']
corpus['diff_a_b'] = df['diff_a_b']
corpus['len_a_prompt'] = df['len_a_prompt']
corpus['len_b_prompt'] = df['len_b_prompt']

Перекодируем целевую переменную из трех столбцов в один. Добавим информацию о длине запроса и ответов.

In [ ]:
corpus.head()

,prompt,a,b,all,target,len_prompt,len_a,len_b,diff_a_b,len_a_prompt,len_b_prompt
0,morally right try certain percentage female m...,question morally right aim certain percentage...,ai personal belief opinion tell question gend...,morally right try certain percentage female m...,0,165,4538,1206,0.734244,27.503030,7.309091
1,difference marriage license marriage certific...,marriage license legal document allow couple ...,marriage license marriage certificate differe...,difference marriage license marriage certific...,1,200,3114,3649,-0.171805,15.570000,18.245000
2,explain function call function,function call process invoke execute function...,function call process invoke function program...,explain function call function function call...,2,60,921,1835,-0.992400,15.350000,30.583333
3,create test set rare category want build clas...,create test set rare category challenge possi...,build classifier rare category create test se...,create test set rare category want build clas...,0,87,3182,1562,0.509114,36.574713,17.954023
4,good way travel tel aviv jerusalem car bus pl...,good way travel tel aviv jerusalem depend per...,good way travel tel aviv jerusalem depend per...,good way travel tel aviv jerusalem car bus pl...,1,79,1300,772,0.406154,16.455696,9.772152


Выделим целевую переменную и остальные признаки для построения модели.

In [ ]:
target = corpus['target']

In [ ]:
features = corpus[['all', 'len_prompt', 'len_a', 'len_b', 'diff_a_b', 'len_a_prompt', 'len_b_prompt']]

Разделим выборку на тренировочную и тестовую (валидационную) в соотношении 80% и 20%.

In [ ]:
features_train, features_test, target_train, target_test \
= train_test_split(features, target, test_size=0.2, random_state=RANDOM_STATE)

Преобразуем текстовую информацию в числовую с помощью CountVectorizer, который считает встречаемость слов в тексте. Не будем учитывать слишком встречающиеся слова, а также очень редкие, они не дадут полезной информации, только будут усложнять анализ.

In [ ]:
count_vectorizer = CountVectorizer(min_df=0.2, max_df=0.8, ngram_range=(1,3)) #, ngram_range=(1,2))
count_vectorizer_fit = count_vectorizer.fit(features_train['all'])
features_train_count_vectorizer = count_vectorizer.transform(features_train['all'])
features_test_count_vectorizer = count_vectorizer.transform(features_test['all'])

print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(45981, 7)
(45981,)
(11496, 7)
(11496,)


В результате получается разреженная матрица. Так как есть другие признаки для анализа, кроме текстовых данных, преобразуем переменные, связанные с длиной запроса и ответов, также в разреженную матрицу. Соединим их с текстовой информацией.

In [ ]:
features_train_count_vectorizer

<45981x26 sparse matrix of type '<class 'numpy.int64'>'
	with 317720 stored elements in Compressed Sparse Row format>

In [ ]:
features_train_other = features_train[['len_prompt', 'len_a', 'len_b', 'diff_a_b', 'len_a_prompt', 'len_b_prompt']]

In [ ]:
features_train = sparse.hstack([features_train_count_vectorizer,features_train_other]).toarray()

In [ ]:
features_test_other = features_test[['len_prompt', 'len_a', 'len_b', 'diff_a_b', 'len_a_prompt', 'len_b_prompt']]

In [ ]:
features_test = sparse.hstack([features_test_count_vectorizer,features_test_other]).toarray()

In [ ]:
features_train.shape

(45981, 32)

Для расчета результатов предсказаний используется метрика log loss. Лучшее качество на данной метрике показала модель XGBClassifier. Подберем оптимальные параметры и выведем величину log loss.

In [ ]:
%%time

xgb_model = RandomizedSearchCV(estimator=xgb.XGBClassifier(random_state=RANDOM_STATE), param_distributions={
    'n_estimators': range(10, 400),
    'learning_rate': [0.01, 0.05],
    'subsample': [0.3, 0.9],
    'max_depth': range(2, 15),
    'colsample_bytree': [0.4, 0.5],
    'min_child_weight': range(1, 10)
}, scoring='neg_log_loss', random_state=RANDOM_STATE, cv=5)

xgb_model_fit = xgb_model.fit(features_train, target_train)
xgb_model_log_loss = xgb_model_fit.best_score_

print('log_loss, xgb:', abs(xgb_model_log_loss))
print('Оптимальные значения параметров:', xgb_model_fit.best_params_)

log_loss, xgb: 1.049803330012377
Оптимальные значения параметров: {'subsample': 0.9, 'n_estimators': 322, 'min_child_weight': 8, 'max_depth': 9, 'learning_rate': 0.01, 'colsample_bytree': 0.5}
CPU times: user 16min 46s, sys: 4.74 s, total: 16min 51s
Wall time: 4min 20s


In [ ]:
xgb_predictions = xgb_model.predict_proba(features_test)
xgb_log_loss = log_loss(target_test, xgb_predictions)

На тестовой (валидационной) выборке значение log loss равно 1.052, что немногим больше, чем на тренировочной. Это показывает, что на результаты модели можно положиться.

In [ ]:
print(xgb_log_loss)

1.0518522450028014


In [ ]:
del features_train
del features_test
del target_train
del target_test

Загрузим тестовую выборку, для которой будем предсказывать, какой из ботов лучше справился с ответом. Всего нам доступно 3 наблюдения.

In [ ]:
df_test = pd.read_csv(test_path, sep=',')

In [ ]:
df_test['len_prompt'] = df_test['prompt'].apply(lambda x: len(x))
df_test['len_a'] = df_test['response_a'].apply(lambda x: len(x))
df_test['len_b'] = df_test['response_b'].apply(lambda x: len(x))

df_test['diff_a_b'] = (df_test['len_a'] - df_test['len_b']) / (df_test['len_a'])
df_test['len_a_prompt'] = df_test['len_a'] / df_test['len_prompt']
df_test['len_b_prompt'] = df_test['len_b'] / df_test['len_prompt']

In [ ]:
df_test.head()

,id,prompt,response_a,response_b,len_prompt,len_a,len_b,diff_a_b,len_a_prompt,len_b_prompt
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran...",86,31,114,-2.677419,0.360465,1.325581
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ...",488,1457,460,0.684283,2.985656,0.942623
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p...",217,3984,3716,0.067269,18.359447,17.124424


Сделаем преобразования, как в тренировочной базе данных, для получения прогноза.

In [ ]:
corpus_prompt_test = df_test['prompt'].values

for i in range(len(corpus_prompt_test)):
    corpus_prompt_test[i] = nlp(corpus_prompt_test[i])
    corpus_prompt_test[i] = [token for token in corpus_prompt_test[i] if not token.is_stop]
    corpus_prompt_test[i] = " ".join([token.lemma_ for token in corpus_prompt_test[i]])
    corpus_prompt_test[i] = re.sub(r"(?!'``)[\W\d]+|(?![\w])'[^a-zA-Z]", ' ', corpus_prompt_test[i]).lower()

In [ ]:
corpus_a_test = df_test['response_a'].values

for i in range(len(corpus_a_test)):
    corpus_a_test[i] = nlp(corpus_a_test[i])
    corpus_a_test[i] = [token for token in corpus_a_test[i] if not token.is_stop]
    corpus_a_test[i] = " ".join([token.lemma_ for token in corpus_a_test[i]])
    corpus_a_test[i] = re.sub(r"(?!'``)[\W\d]+|(?![\w])'[^a-zA-Z]", ' ', corpus_a_test[i]).lower()

In [ ]:
corpus_b_test = df_test['response_b'].values

for i in range(len(corpus_b_test)):
    corpus_b_test[i] = nlp(corpus_b_test[i])
    corpus_b_test[i] = [token for token in corpus_b_test[i] if not token.is_stop]
    corpus_b_test[i] = " ".join([token.lemma_ for token in corpus_b_test[i]])
    corpus_b_test[i] = re.sub(r"(?!'``)[\W\d]+|(?![\w])'[^a-zA-Z]", ' ', corpus_b_test[i]).lower()

In [ ]:
corpus_prompt_test = pd.DataFrame(corpus_prompt_test)
corpus_a_test = pd.DataFrame(corpus_a_test)
corpus_b_test = pd.DataFrame(corpus_b_test)

In [ ]:
corpus_test = pd.merge(corpus_prompt_test, corpus_a_test, left_index=True, right_index=True)
corpus_test = pd.merge(corpus_test, corpus_b_test, left_index=True, right_index=True)
corpus_test.columns = ['prompt', 'a', 'b']

In [ ]:
corpus_test['all'] = corpus_test['prompt'] + corpus_test['a'] + corpus_test['b']


In [ ]:
corpus_test['len_prompt'] = df_test['len_prompt']
corpus_test['len_a'] = df_test['len_a']
corpus_test['len_b'] = df_test['len_b']
corpus_test['diff_a_b'] = df_test['diff_a_b']
corpus_test['len_a_prompt'] = df_test['len_a_prompt']
corpus_test['len_b_prompt'] = df_test['len_b_prompt']

In [ ]:
corpus_test.head()

,prompt,a,b,all,len_prompt,len_a,len_b,diff_a_b,len_a_prompt,len_b_prompt
0,orange today eat orange yesterday orange,orange today,orange eat orange yesterday affect number ora...,orange today eat orange yesterday orange ora...,86,31,114,-2.677419,0.360465,1.325581
1,mediator heated political debate oppose party...,thank share detail situation mediator underst...,mr reddy ms blue valid point argument hand mr...,mediator heated political debate oppose party...,488,1457,460,0.684283,2.985656,0.942623
2,initialize classification head transfer learn...,want initialize classification head transfer ...,initialize classification head perform transf...,initialize classification head transfer learn...,217,3984,3716,0.067269,18.359447,17.124424


In [ ]:
features_submit_count_vectorizer = corpus_test['all']
features_submit_count_vectorizer = count_vectorizer.transform(features_submit_count_vectorizer)

In [ ]:
print(features_submit_count_vectorizer.shape)

(3, 26)


In [ ]:
features_submit_other = corpus_test[['len_prompt', 'len_a', 'len_b', 'diff_a_b', 'len_a_prompt', 'len_b_prompt']]

In [ ]:
features_submit = sparse.hstack([features_submit_count_vectorizer,features_submit_other]).toarray()

Используем уже настроенную модель XGBClassifier и сделаем предсказания. Оформим результаты в том виде, какой требуется для загрузки.

In [ ]:
xgb_predictions_submit = xgb_model.predict_proba(features_submit)

In [ ]:
sample_submission = pd.DataFrame(xgb_predictions_submit)

In [ ]:
sample_submission.columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

In [ ]:
sample_submission['id'] = df_test['id']

In [ ]:
sample_submission = sample_submission[['id'] + [x for x in sample_submission.columns if x != 'id']]

In [ ]:
sample_submission

,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.251833,0.278688,0.469479
1,211333,0.408196,0.277460,0.314343
2,1233961,0.383100,0.306598,0.310303


In [ ]:
sample_submission.to_csv("submission.csv", index=False)

Итоговая метрика на тестовой выборке 1.053. На закрытой части тестовой выборки (Private Score) - на основе которой рассчитывались призовые места - 1.088. Метрика участника, занявшего первое место в соревновании - 0.969.

**Вывод:** для прогнозирования предпочтений пользователей была проанализирована текстовая информация и числовые данные. Текстовые данные обработаны (проведена лемматизация, очистка текста от стоп-слов, удаление символов) и приведены к числовому виду (с помощью CountVectorizer). Числовые данные - характеристики длины текстовых запросов и ответов.

С помощью RandomizedSearchCV были подобраны оптимальные параметры модели XGBClassifier на метрике log loss.

Без использования нейросетей получилось добиться достаточно высокого качества предсказания за сравнительно небольшое количество времени. При подготовке модели бОльшую часть времени заняла обработка текстовых данных (43 минуты), подбор гиперпараметров - 4 минуты.